In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_10%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class
0,1.49,0.0,NaN,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,NaN,0.01,0.0,0.00,0.0,0.00,23.0,1
1,0.73,0.0,0.35,0.38,NaN,5.86,0.64,2.06,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0,1
2,1.22,0.0,0.73,0.67,NaN,8.72,0.88,4.35,5.17,NaN,...,211.44,1431.0,NaN,0.01,0.0,10.66,1.0,0.18,23.0,1
3,2.50,0.0,1.16,NaN,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0,1
4,NaN,0.0,1.72,NaN,5.15,9.11,1.04,3.10,4.93,12.82,...,NaN,117.0,114.0,0.00,0.0,18.18,NaN,13.32,23.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0
1036,1.76,0.0,NaN,0.69,2.43,NaN,0.74,2.72,3.93,NaN,...,143.80,47.0,44.0,0.01,0.0,NaN,NaN,25.79,1.0,0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,NaN,...,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0,0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         110
Jitter (local, absolute)               119
Jitter (rap)                           110
Jitter (ppq5)                          112
Jitter (ddp)                           112
Shimmer (local)                        106
Shimmer (local, dB)                    101
Shimmer (apq3)                         100
Shimmer (apq5)                          99
Shimmer (apq11)                         94
Shimmer (dda)                          106
AC                                     110
NTH                                    110
HTN                                    111
Median pitch                           119
Mean pitch                             112
Standard deviation                     103
Minimum pitch                          125
Maximum pitch                          119
Number of pulses                       102
Number of periods                      111
Mean period                            113
Standard deviation of period           100
Fraction of

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 2912
persentase : 10.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         10.576923
Jitter (local, absolute)               11.442308
Jitter (rap)                           10.576923
Jitter (ppq5)                          10.769231
Jitter (ddp)                           10.769231
Shimmer (local)                        10.192308
Shimmer (local, dB)                     9.711538
Shimmer (apq3)                          9.615385
Shimmer (apq5)                          9.519231
Shimmer (apq11)                         9.038462
Shimmer (dda)                          10.192308
AC                                     10.576923
NTH                                    10.576923
HTN                                    10.673077
Median pitch                           11.442308
Mean pitch                             10.769231
Standard deviation                      9.903846
Minimum pitch                          12.019231
Maximum pitch                          11.442308
Number of pulses                        9.807692
Number of periods   

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,9)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
0,1.49,0.0,0.00,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,142.23,187.58,160.0,0.0,0.01,0.0,0.00,0.0,0.00,23.0
1,0.73,0.0,0.35,0.38,0.00,5.86,0.64,2.06,3.18,7.19,...,159.52,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0
2,1.22,0.0,0.73,0.67,0.00,8.72,0.88,4.35,5.17,0.00,...,0.00,211.44,1431.0,0.0,0.01,0.0,10.66,1.0,0.18,23.0
3,2.50,0.0,1.16,0.00,3.47,13.51,0.00,5.26,8.77,16.78,...,182.71,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0
4,0.00,0.0,1.72,0.00,5.15,9.11,1.04,3.10,4.93,12.82,...,182.82,0.00,117.0,114.0,0.00,0.0,18.18,0.0,13.32,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,96.15,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0
1036,1.76,0.0,0.00,0.69,2.43,0.00,0.74,2.72,3.93,0.00,...,121.49,143.80,47.0,44.0,0.01,0.0,0.00,0.0,25.79,1.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,0.00,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,0.00,...,90.05,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 5922


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)

Kolom : 1
[0.0, 2.68, 2.16, 4.34, 2.57, 0.89, 2.33, 0.0, 1.74]
mean :  1.8566666666666667
standar deviasi :  1.3128510112643315
fuzzy :  [(0.0, 0.36791165763565825), (2.68, 0.8214961277910431), (2.16, 0.9736638806890527), (4.34, 0.16715903357279127), (2.57, 0.862778277501698), (0.89, 0.7625792416964177), (2.33, 0.9370793961989641), (0.0, 0.36791165763565825), (1.74, 0.9960596878331766)]
fuzzy :  None
pembilang :  11.842768358736754
penyebut :  6.256638960554461
rata-rata tertimbang :  1.8928323071541358
 
[2.11, 2.29, 1.29, 2.5, 2.72, 3.05, 1.53, 1.92, 2.48]
mean :  2.21
standar deviasi :  0.5304086474910956
fuzzy :  [(2.11, 0.9823862980429209), (2.29, 0.9886912024315824), (1.29, 0.22221510291280475), (2.5, 0.8611803529848387), (2.72, 0.6298868343359834), (3.05, 0.28539005710682064), (1.53, 0.4396766361764541), (1.92, 0.8611803529848387), (2.48, 0.8784922635078348)]
fuzzy :  None
pembilang :  13.865110515808475
penyebut :  6.1490991004840785
rata-rata tertimbang :  2.254819818193687
 


[0.45, 2.16, 4.34, 0.51, 1.76, 0.28, 0.65, 0.46, 0.43]
mean :  1.2266666666666668
standar deviasi :  1.26563115567776
fuzzy :  [(0.45, 0.8283914179330173), (2.16, 0.7619423738734851), (4.34, 0.04854546977172734), (0.51, 0.8518845536078522), (1.76, 0.9150481761253009), (0.28, 0.7560044913285487), (0.65, 0.9014135313022418), (0.46, 0.8323913479088013), (0.43, 0.8202954998887936)]
fuzzy :  None
pembilang :  5.807432054675098
penyebut :  6.715916861739768
rata-rata tertimbang :  0.8647266150299951
 
[2.91, 3.67, 1.81, 3.46, 2.14, 1.41, 2.56, 2.49, 0.0]
mean :  2.2722222222222226
standar deviasi :  1.0561542161990654
fuzzy :  [(2.91, 0.8333434631942067), (3.67, 0.4165776999953471), (1.81, 0.9086845173058352), (3.46, 0.5313547026144984), (2.14, 0.9921948939712459), (1.41, 0.7166248040832682), (2.56, 0.963562408495878), (2.49, 0.9789675556694728), (0.0, 0.09886049601824148)]
fuzzy :  None
pembilang :  15.4751629104701
penyebut :  6.440170541347995
rata-rata tertimbang :  2.402911974320324
 
[

[0.0, 0.2, 0.35, 0.67, 0.98, 0.32, 0.49454217892120683, 0.39, 0.66]
mean :  0.4516157976579119
standar deviasi :  0.2730202492458525
fuzzy :  [(0.0, 0.2546257811077741), (0.2, 0.6540114748886076), (0.35, 0.9330877326623285), (0.67, 0.7262412055310951), (0.98, 0.15373016535049672), (0.32, 0.8903102069666762), (0.49454217892120683, 0.9877170452951686), (0.39, 0.9748578859173689), (0.66, 0.7473293714119595)]
fuzzy :  None
pembilang :  2.7414191377657495
penyebut :  6.321910869131475
rata-rata tertimbang :  0.43363773936635314
 
[0.0, 1.59, 3.85, 5.88, 2.33, 0.0, 3.36, 4.51, 3.64]
mean :  2.795555555555556
standar deviasi :  1.882581626449943
fuzzy :  [(0.0, 0.33206102043026997), (1.59, 0.8146327275333116), (3.85, 0.8548383794285214), (5.88, 0.261307376276671), (2.33, 0.9698881041193763), (0.0, 0.33206102043026997), (3.36, 0.9560522039921386), (4.51, 0.6605824743907345), (3.64, 0.9043027372983605)]
fuzzy :  None
pembilang :  17.865944781364576
penyebut :  6.085726043899654
rata-rata tertim

[0.0, 3.52, 3.78, 1.67, 0.0, 4.38, 3.38, 2.33, 2.38]
mean :  2.3822222222222216
standar deviasi :  1.4942465792184945
fuzzy :  [(0.0, 0.2806335112239694), (3.52, 0.7483648162037557), (3.78, 0.6456624204293933), (1.67, 0.8926305399977285), (0.0, 0.2806335112239694), (4.38, 0.40915342160998164), (3.38, 0.8001792504042975), (2.33, 0.9993895381406898), (2.38, 0.9999988942545416)]
fuzzy :  None
pembilang :  15.770813949268392
penyebut :  6.056645903488326
rata-rata tertimbang :  2.603885748081325
 
[0.0, 5.29, 10.44, 2.82, 2.66, 5.88, 1.15, 2.88, 2.33]
mean :  3.7166666666666663
standar deviasi :  2.93074507022816
fuzzy :  [(0.0, 0.44751861575117124), (5.29, 0.8658165626835986), (10.44, 0.07199886244736997), (2.82, 0.9542797680287773), (2.66, 0.9370770634779126), (5.88, 0.761545440328885), (1.15, 0.6815067030653313), (2.88, 0.960073962837401), (2.33, 0.8941145542557626)]
fuzzy :  None
pembilang :  20.625451497285795
penyebut :  6.57393153287621
rata-rata tertimbang :  3.1374606495577235
 
[

[0.0, 2.33, 2.16, 2.44, 1.98, 2.57, 3.01, 4.34, 0.89]
mean :  2.191111111111111
standar deviasi :  1.1572231557045363
fuzzy :  [(0.0, 0.1665705257599832), (2.33, 0.9928243228284269), (2.16, 0.9996387204965863), (2.44, 0.9771392920084467), (1.98, 0.9834992022678795), (2.57, 0.9478170397808264), (3.01, 0.7785313456251173), (4.34, 0.178364481918653), (0.89, 0.5315257983065258)]
fuzzy :  None
pembilang :  14.83047755604196
penyebut :  6.555910728992445
rata-rata tertimbang :  2.2621536761409202
 
[0.0, 1.46, 1.17, 2.05, 1.5, 1.7, 2.14, 1.52, 2.3894508357365773]
mean :  1.5477167595262862
standar deviasi :  0.6568976570834296
fuzzy :  [(0.0, 0.062329746278164413), (1.46, 0.9911251628104807), (1.17, 0.8476428573373019), (2.05, 0.7465454905642462), (1.5, 0.9973654968132898), (1.7, 0.9734898284550395), (2.14, 0.6660220470341112), (1.52, 0.9991103458748669), (2.3894508357365773, 0.4400445025243717)]
fuzzy :  None
pembilang :  11.115583700739094
penyebut :  6.723675477691873
rata-rata tertimbang

...

[8.0, 46.0, 8.0, 46.0, 1.0, 8.0, 1.0, 8.0, 8.0]
mean :  14.88888888888889
standar deviasi :  16.861820413416844
fuzzy :  [(8.0, 0.9199391907302169), (46.0, 0.18232742391813664), (8.0, 0.9199391907302169), (46.0, 0.18232742391813664), (1.0, 0.7123426279475376), (8.0, 0.9199391907302169), (1.0, 0.7123426279475376), (8.0, 0.9199391907302169), (8.0, 0.9199391907302169)]
fuzzy :  None
pembilang :  54.99637588557231
penyebut :  6.389036057382432
rata-rata tertimbang :  8.607930115220567
 
[1.0, 32.0, 16.0, 1.0, 11.0, 5.0, 5.0, 1.0, 24.0]
mean :  10.666666666666666
standar deviasi :  10.572501018102471
fuzzy :  [(1.0, 0.6583959976685729), (32.0, 0.13060546608785148), (16.0, 0.8805369491624729), (1.0, 0.6583959976685729), (11.0, 0.9995031571076212), (5.0, 0.8662140085131994), (5.0, 0.8662140085131994), (1.0, 0.6583959976685729), (24.0, 0.4515152296205452)]
fuzzy :  None
pembilang :  50.736194418625445
penyebut :  6.169776812010609
rata-rata tertimbang :  8.223343560800787
 
[1.0, 5.0, 11.0, 1.

[0.0, 16.0, 0.0, 1.0, 32.0, 7.333119406853483, 55.0, 1.0, 11.0]
mean :  13.703679934094831
standar deviasi :  17.61204463683921
fuzzy :  [(0.0, 0.7388374913165299), (16.0, 0.9915369672490092), (0.0, 0.7388374913165299), (1.0, 0.7709633324118779), (32.0, 0.5830096654544857), (7.333119406853483, 0.936681008060131), (55.0, 0.06401070486666033), (1.0, 0.7709633324118779), (11.0, 0.9882872176207574)]
fuzzy :  None
pembilang :  57.323369275082925
penyebut :  6.58312721070786
rata-rata tertimbang :  8.707619865197644
 
[0.0, 32.0, 1.0, 1.0, 8.0, 0.0, 11.0, 1.0, 1.0]
mean :  6.111111111111111
standar deviasi :  9.87076991756125
fuzzy :  [(0.0, 0.8256123703883791), (32.0, 0.03209230265640885), (1.0, 0.8745499152758507), (1.0, 0.8745499152758507), (8.0, 0.9818587871279747), (0.0, 0.8256123703883791), (11.0, 0.8845792802580354), (1.0, 0.8745499152758507), (1.0, 0.8745499152758507)]
fuzzy :  None
pembilang :  22.11039572597067
penyebut :  7.04795477192258
rata-rata tertimbang :  3.1371364376589597

[0.0, 1.0, 1.0, 1.0, 1.0, 46.0, 0.0, 1.0, 1.0]
mean :  5.777777777777778
standar deviasi :  14.226561837928202
fuzzy :  [(0.0, 0.92084767683804), (1.0, 0.9451735329189382), (1.0, 0.9451735329189382), (1.0, 0.9451735329189382), (1.0, 0.9451735329189382), (46.0, 0.01838368455980099), (0.0, 0.92084767683804), (1.0, 0.9451735329189382), (1.0, 0.9451735329189382)]
fuzzy :  None
pembilang :  6.516690687264474
penyebut :  7.53112023574951
rata-rata tertimbang :  0.8653016394998402
 
[0.0, 1.0, 1.0, 11.0, 11.0, 1.0, 55.0, 11.0, 1.0]
mean :  10.222222222222221
standar deviasi :  16.50439522455746
fuzzy :  [(0.0, 0.8254836937119734), (1.0, 0.8554754819885209), (1.0, 0.8554754819885209), (11.0, 0.9988903251852287), (11.0, 0.9988903251852287), (1.0, 0.8554754819885209), (55.0, 0.025222334242662348), (11.0, 0.9988903251852287), (1.0, 0.8554754819885209)]
fuzzy :  None
pembilang :  37.772511042413065
penyebut :  7.269278931464404
rata-rata tertimbang :  5.196184023000993
 
[0.0, 1.0, 16.0, 16.0, 1.0

[0.0, 11.0, 1.0, 1.0, 20.0, 1.0, 16.0, 1.0, 0.0]
mean :  5.666666666666667
standar deviasi :  7.393691004272944
fuzzy :  [(0.0, 0.7455234163524656), (11.0, 0.770947017592218), (1.0, 0.8194135813049688), (1.0, 0.8194135813049688), (20.0, 0.15276259777667817), (1.0, 0.8194135813049688), (16.0, 0.3766176656867444), (1.0, 0.8194135813049688), (0.0, 0.7455234163524656)]
fuzzy :  None
pembilang :  20.839206125255746
penyebut :  6.069028438980446
rata-rata tertimbang :  3.433697227616977
 
[0.0, 55.0, 0.0, 16.569273434098054, 1.0, 55.0, 55.0, 1.0, 12.0]
mean :  21.72991927045534
standar deviasi :  24.142715254248138
fuzzy :  [(0.0, 0.6669681337094422), (55.0, 0.38696363762601543), (0.0, 0.6669681337094422), (16.569273434098054, 0.9774155291830655), (1.0, 0.6917063574730877), (55.0, 0.38696363762601543), (55.0, 0.38696363762601543), (1.0, 0.6917063574730877), (12.0, 0.9220066520761008)]
fuzzy :  None
pembilang :  92.49155790991979
penyebut :  5.777662076502272
rata-rata tertimbang :  16.008474

[0.0, 1.0, 0.0, 1.0, 40.0, 23.0, 1.0, 1.0, 1.0]
mean :  7.555555555555555
standar deviasi :  13.417328026027562
fuzzy :  [(0.0, 0.8533929240908479), (1.0, 0.8874996885212628), (0.0, 0.8533929240908479), (1.0, 0.8874996885212628), (40.0, 0.053755928948486485), (23.0, 0.5155988870996281), (1.0, 0.8874996885212628), (1.0, 0.8874996885212628), (1.0, 0.8874996885212628)]
fuzzy :  None
pembilang :  18.446510003837222
penyebut :  6.713639106836125
rata-rata tertimbang :  2.747617158189836
 
[0.0, 1.0, 20.0, 46.0, 8.0, 46.0, 1.0, 1.0, 20.0]
mean :  15.88888888888889
standar deviasi :  17.722831755453154
fuzzy :  [(0.0, 0.6690923513532483), (1.0, 0.7026864704781165), (20.0, 0.9734570815753896), (46.0, 0.23618195728735386), (8.0, 0.9056899974893323), (46.0, 0.23618195728735386), (1.0, 0.7026864704781165), (1.0, 0.7026864704781165), (20.0, 0.9734570815753896)]
fuzzy :  None
pembilang :  70.02060272480115
penyebut :  6.102119838002418
rata-rata tertimbang :  11.47479967350543
 
[0.0, 1.0, 1.0, 2.7

In [15]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
0,1.490000,0.0,0.825948,0.790000,2.700000,8.330000,0.780000,4.52,4.610000,6.800000,...,142.230000,187.580000,160.0,169.836013,0.010000,0.0,6.343545,0.004864,1.086205,23.0
1,0.730000,0.0,0.350000,0.380000,1.595224,5.860000,0.640000,2.06,3.180000,7.190000,...,159.520000,234.510000,170.0,169.000000,0.010000,0.0,2.250000,0.395565,18.767157,23.0
2,1.220000,0.0,0.730000,0.670000,0.992549,8.720000,0.880000,4.35,5.170000,5.370198,...,131.663164,211.440000,1431.0,1019.273603,0.010000,0.0,10.660000,1.000000,0.180000,23.0
3,2.500000,0.0,1.160000,1.209419,3.470000,13.510000,0.912503,5.26,8.770000,16.780000,...,182.710000,220.230000,94.0,92.000000,0.001911,0.0,13.174269,1.000000,0.062945,23.0
4,1.892832,0.0,1.720000,0.997087,5.150000,9.110000,1.040000,3.10,4.930000,12.820000,...,182.820000,236.990526,117.0,114.000000,0.001911,0.0,18.180000,0.054180,13.320000,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.130000,0.0,2.720000,2.510000,8.160000,9.850000,0.970000,4.59,6.810000,8.300000,...,96.150000,158.590000,63.0,60.000000,0.010000,0.0,24.190000,2.000000,15.110000,1.0
1036,1.760000,0.0,1.163164,0.690000,2.430000,9.203162,0.740000,2.72,3.930000,8.530039,...,121.490000,143.800000,47.0,44.000000,0.010000,0.0,19.399698,1.104939,25.790000,1.0
1037,2.910000,0.0,1.140000,1.270000,3.420000,15.490000,1.460000,6.43,10.140000,14.240000,...,101.029392,149.640000,79.0,77.000000,0.010000,0.0,20.000000,1.000000,4.570000,1.0
1038,3.790000,0.0,1.790000,1.480000,5.360000,15.700000,1.370000,6.37,11.090000,11.749470,...,90.050000,451.370000,75.0,70.000000,0.010000,0.0,16.670000,3.000000,19.580000,1.0


In [16]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1040.000000,1.040000e+03,1040.00000,1040.000000,1.040000e+03,1040.000000,1040.000000
mean,2.638054,0.0,1.216531,1.322631,3.637090,12.774364,1.177584,5.591096,7.869562,12.047508,...,133.621897,232.384010,108.561662,104.187845,7.912795e-03,0.00001,28.238592,1.244400e+00,13.935225,12.562363
std,1.708331,0.0,0.910810,1.103547,2.805332,5.294813,0.404599,2.866496,4.713904,5.819496,...,45.724469,119.555313,149.664501,142.398701,3.825668e-03,0.00031,19.439364,1.497205e+00,13.332970,15.319367
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,67.960000,85.540000,6.000000,4.000000,5.278066e-11,0.00000,0.160000,1.785356e-07,0.000222,0.815164
25%,1.540000,0.0,0.639585,0.680000,1.903449,9.475000,0.940000,3.720000,5.255000,8.420000,...,101.240000,143.055000,42.000000,41.000000,9.232496e-03,0.00000,11.425718,3.134472e-01,3.412879,1.000000
50%,2.359617,0.0,1.014769,1.100000,3.031062,12.080000,1.160000,5.065000,6.935000,11.300000,...,126.430000,195.205000,64.000000,61.000000,1.000000e-02,0.00000,26.370000,1.000000e+00,9.550000,5.000000
75%,3.332500,0.0,1.560000,1.620000,4.635000,15.073149,1.380000,6.742500,9.220000,14.845147,...,157.597278,260.825944,111.000000,108.000000,1.000000e-02,0.00000,41.670000,1.000608e+00,21.452500,23.000000
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,44.760000,...,452.080000,526.410000,1490.000000,1483.000000,1.000000e-02,0.01000,88.160000,1.200000e+01,69.120000,55.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.values
y = df.iloc[:, -1].values

In [100]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=17)

In [101]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [102]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[137   7]
 [  7 109]]
0.9461538461538461
